### Following your goal

In the previous section, you assumed that you were able to let KITT follow any predefined trajectory. You will now design a controller which keeps KITT on track.

Intuitively, you will think of a solution where you know your current position/orientation, and you always steer towards the target. Once you are oriented towards the target, your "angle error" is zero, and you just have to drive straight. In all other cases, the angle error determines how much you need to steer. It is easy to see that this approach might work, but also might become unstable once you are very close to the target.

Suppose KITT is driving on its trajectory $S$, where its orientation with respect to the $x$-axis is given by $\theta$. Let the angle tangent to its trajectory $S$ be given by $\theta_t$. Ideally, this angle should be equal to KITT's orientation. If KITT's orientation deviates from this angle, KITT should turn its wheels to steer back. This motivates a feedback law given by

$$
\phi = -k(\theta - \theta_t)
$$

for a positive $k$. Substituting in Equation (\ref{eq:steering-derivative}) (@B update) yields the autonomous non-linear system

$$
L \,\frac{d}{dt}\theta \;+\; v \sin(k(\theta-\theta_t)) = 0\,.
$$

You should choose $k$ such that this system is asymptotically stable. To investigate the stability, you introduce a potential function (error function) given by

$$
T(\theta) = \frac{1}{2}(\theta-\theta_t)^2 \,.
$$

A first observation is that $T(\theta)=0$ if and only if $\theta=\theta_t$, which is exactly what you want. Second, notice that $T(\theta) \ge 0$. you can conclude that $T$'s minimum corresponds to our equilibrium point. Notice that

$$
\frac{d}{dt} T(\theta) = -(\theta-\theta_t)\frac{v \sin{(k(\theta-\theta_t))}}{L}.
$$

Figure \ref{fig:KITTpotential-figure}  (@B update) depicts both \(T(\theta)\) and its time-derivative.



<img src="pictures/potentialfunction.jpg" alt="KITTpotential-figure" width="250px">

*Graph of potential function and it's derivative*
<!-- 
```{figure} potentialfunction.jpg
---
height/width: 150px
name: KITTpotential-figure
---
Graph of potential function and it's derivative
```
-->

Consider KITT's orientation at any instant. If $ |\theta| < \theta_m $, then by Figure \ref{fig:lyaponuv} the potential function will have a negative derivative. But then it will decrease over time and will keep decreasing until $\theta = \theta_t$. So in conclusion, if $ |\theta| < \theta_m $, then $\theta$ will converge to its equilibrium point. By inspection of Equation (\ref{eq:lyapunov}) you can now state that Equation (\ref{eq:angle-diffeq}) is locally asymptotically stable for any

$$
-\frac{\pi}{k} < \theta - \theta_t < \frac{\pi}{k}.
$$

This treatment is also called the investigation of *Lyapunov stability*, where $T$ is called the *Lyapunov function*. It is an extensive topic in the control literature.